## Setup and Import Libraries

In [ ]:
import os
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.schema import Document
from langchain.chat_models import init_chat_model
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

In [2]:
load_dotenv()

True

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

In [5]:
embeddings = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")
llm = init_chat_model(model="groq:openai/gpt-oss-20b")

## Create Documents

In [7]:
documents = [
    Document(page_content="LangChain helps build LLM application"),
    Document(page_content="Pinecone is a vector database for semantic search"),
    Document(page_content="The Eiffel Tower is located in Paris"),
    Document(page_content="LangChain can be used to develop Agentic AI applications"),
    Document(page_content="LangChain has many types of retrievers"),
]

documents

[Document(metadata={}, page_content='LangChain helps build LLM application'),
 Document(metadata={}, page_content='Pinecone is a vector database for semantic search'),
 Document(metadata={}, page_content='The Eiffel Tower is located in Paris'),
 Document(metadata={}, page_content='LangChain can be used to develop Agentic AI applications'),
 Document(metadata={}, page_content='LangChain has many types of retrievers')]

## Create Dense Retriever

In [8]:
dense_vector_store = FAISS.from_documents(
    documents=documents,
    embedding=embeddings
)

dense_retriever = dense_vector_store.as_retriever()

## Create Sparse Retriever

In [10]:
sparse_retriever = BM25Retriever.from_documents(
    documents=documents
)

sparse_retriever.k = 3

## Combine with Ensemble Retriever

In [11]:
hybrid_retriever = EnsembleRetriever(
    retrievers=[dense_retriever, sparse_retriever],
    weights=[0.7, 0.3]
)

In [12]:
hybrid_retriever

EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FB245BCEF0>, search_kwargs={}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x000001FB25AFBC50>, k=3)], weights=[0.7, 0.3])

## Query Hybrid Retriever

In [13]:
query = "How can I build an application using LLMs?"

results = hybrid_retriever.invoke(input=query)

for i, result in enumerate(results):
    print(f"\n 🔶 Document {i + 1}:\n{result.page_content}")


 🔶 Document 1:
LangChain helps build LLM application

 🔶 Document 2:
LangChain can be used to develop Agentic AI applications

 🔶 Document 3:
LangChain has many types of retrievers

 🔶 Document 4:
Pinecone is a vector database for semantic search


## Create RAG Pipeline with Hybrid Retriever

In [ ]:
prompt = PromptTemplate.from_template(""" 
Answer the question based on the context below.
                                      
Context: {context}
                                      
Question: {input}
""")

In [15]:
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

In [16]:
rag_chain = create_retrieval_chain(
    retriever=hybrid_retriever,
    combine_docs_chain=document_chain
)

rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FB245BCEF0>, search_kwargs={}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x000001FB25AFBC50>, k=3)], weights=[0.7, 0.3]), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template=' \nAnswer the question based on the context below.\n\nContext: {context}\n\nQuestion: {input}\n')
            | ChatGroq(client=<g

## Query RAG Chain

In [18]:
query = "How can I build an app using LLMs"

response = rag_chain.invoke({"input": query})

print(f"✅ Answer:\n {response["answer"]}")

print("\n📃Source Documents:")
for i, result in enumerate(response["context"]):
    print(f"\nResult {i+1}: {result.page_content}")

✅ Answer:
 ## Quick‑Start Guide: Building an LLM‑Powered App with LangChain & Pinecone

Below is a practical, step‑by‑step recipe that pulls together everything from the context:

| Step | What you’ll do | Why it matters |
|------|----------------|----------------|
| 1 | **Pick an LLM** (OpenAI, Anthropic, Cohere, etc.) | The core “brain” that generates text. |
| 2 | **Create a LangChain environment** | LangChain gives you reusable building blocks: prompts, chains, agents, and retrievers. |
| 3 | **Add a semantic retriever** (e.g., `VectorStoreRetriever`) | Pulls in relevant documents before the LLM answers. |
| 4 | **Connect to Pinecone** | Fast, scalable vector database for semantic search. |
| 5 | **Wrap everything in an Agent** | Lets the system decide *what* to do (search, ask, write, etc.) in a loop. |
| 6 | **Deploy** (FastAPI + Docker or serverless) | Exposes your app to users. |

---

### 1. Pick Your LLM

```bash
pip install openai  # or anthropic, cohere, etc.
```

```python